#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import pickle
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.models import parameter_learning as lfi
from src.models import inference as inf
from src.visualization import visualize as viz


In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Inference (I)

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/e_ksea_penalised.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/p_fc.csv')

In [5]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*0.8))
test = [x for x in samples if x not in train]
print(test)
# filter data
testing_data = {}
testing_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(test)].reset_index(drop=True)
testing_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(test)].reset_index(drop=True)

['AZD6738', 'Dasatinib', 'GO6983', 'GSK690693', 'KD025', 'KN62', 'MK2206', 'PD153035', 'PF4708671', 'PIK294', 'Tofacitinib', 'Ulixertinib', 'Vemurafenib']


In [6]:
# Adding evidence
model_inference = 'models/ebdt_data/sub_network/p_model/p_model_exp10_testing_200i.pl'

#### Inferring p_occupancy

Adding queries

In [7]:
sample_ids = testing_data['p_fc']['sample'].unique().tolist()
phosphosites = testing_data['p_fc']['phosphosite'].unique().tolist()
queries = inf.generate_queries(schema.POccupancyPredicate, sample_ids=sample_ids, phosphosites=phosphosites)
len(queries)

117

In [8]:
queries[:10]

['query(p_occupancy("ABI1(Y213)", "PD153035", _)).',
 'query(p_occupancy("ANXA2P2(Y24)", "PD153035", _)).',
 'query(p_occupancy("ATF1(S198)", "PD153035", _)).',
 'query(p_occupancy("BCLAF1(Y284)", "PD153035", _)).',
 'query(p_occupancy("HIPK2(Y361)", "PD153035", _)).',
 'query(p_occupancy("PTTG1IP(Y174)", "PD153035", _)).',
 'query(p_occupancy("PXN(Y118)", "PD153035", _)).',
 'query(p_occupancy("PXN(Y88)", "PD153035", _)).',
 'query(p_occupancy("RBM39(Y95)", "PD153035", _)).',
 'query(p_occupancy("ABI1(Y213)", "Tofacitinib", _)).']

Adding evidence (phosphosite and relevant enzymes)

In [9]:
# Mapping data to Problog predicates
predicates = {}
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(testing_data['p_fc'], 'phosphosite', 'sample', 'value')
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(testing_data['e_ksea'], 'enzyme', 'sample', 'value')

In [10]:
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/es_interaction.csv')

In [11]:
evidence_dict = {}
for s in sample_ids:
    evidence_dict[s] = {}
    for p in phosphosites:
        enzymes = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e]))
        evid_generator = build.ProblogStatementGenerator(predicates['p_fc'])
        evidence_p = evid_generator.generate_facts(build.EvidenceTemplate, select=[s, p])
        evidence_dict[s][p] = evidence_e + evidence_p

In [12]:
# make evidence_dict['AZD5438'] into list
evidence_list = []
for s in evidence_dict.keys():
    for p in evidence_dict[s]:
        evidence_list.append(evidence_dict[s][p])
evidence_list[:10]

[['evidence(e_ksea("ABL1", "PD153035", inc)).',
  'evidence(p_fc("ABI1(Y213)", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(p_fc("ANXA2P2(Y24)", "PD153035", inc)).'],
 ['evidence(e_ksea("HIPK2", "PD153035", dec)).',
  'evidence(p_fc("ATF1(S198)", "PD153035", dec)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(p_fc("BCLAF1(Y284)", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(e_ksea("HIPK2", "PD153035", dec)).',
  'evidence(p_fc("HIPK2(Y361)", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(p_fc("PTTG1IP(Y174)", "PD153035", dec)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(e_ksea("ABL1", "PD153035", inc)).',
  'evidence(e_ksea("PTK2", "PD153035", dec)).',
  'evidence(e_ksea("PTPRG", "PD153035", dec)).',
  'evidence(e_ksea("FYN", "PD153035", dec)).',
  'evidence(e_ksea("PTK6", "PD153035", dec)).',
  'evidence(p_fc("PXN(Y118)", "PD153035", dec)).'],
 

In [13]:
inference_output = 'models/ebdt_data/sub_network/p_model/inference_log_10.txt'
inf_data_dict = inf.infer_marginf(model_inference, queries, inference_output, evidence=evidence_list)
pickle.dump(inf_data_dict, open('models/ebdt_data/sub_network/p_model/testing_relepev_exp10_200i.pkl', 'wb'))

['query(p_occupancy("ABI1(Y213)", "PD153035", _)).']
1/117
Inferring data (['query(p_occupancy("ABI1(Y213)", "PD153035", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp10_testing_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_10.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0001s
[INFO] Grounding: 0.0233s
[INFO] Cycle breaking: 0.0020s
[INFO] Clark's completion: 0.0002s
[INFO] DSharp compilation: 0.0222s
[INFO] Total time: 1.7180s
Finished inference (['query(p_occupancy("ABI1(Y213)", "PD153035", _)).'])...
['query(p_occupancy("ANXA2P2(Y24)", "PD153035", _)).']
2/117
Inferring data (['query(p_occupancy("ANXA2P2(Y24)", "PD153035", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp10_testing_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_10.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0000s
[INFO] Grounding: 0.0285s
[INFO] Cycle breaking: 0.0026s
[INFO] Clark's complet

Adding evidence (relevant enzymes only)

In [14]:
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/es_interaction.csv')

In [15]:
evidence_dict = {}
for s in sample_ids:
    evidence_dict[s] = {}
    for p in phosphosites:
        enzymes = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e]))
        evidence_dict[s][p] = evidence_e

In [16]:
# make evidence_dict['AZD5438'] into list
evidence_list = []
for s in evidence_dict.keys():
    for p in evidence_dict[s]:
        evidence_list.append(evidence_dict[s][p])
evidence_list[:10]

[['evidence(e_ksea("ABL1", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).'],
 ['evidence(e_ksea("HIPK2", "PD153035", dec)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(e_ksea("HIPK2", "PD153035", dec)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(e_ksea("ABL1", "PD153035", inc)).',
  'evidence(e_ksea("PTK2", "PD153035", dec)).',
  'evidence(e_ksea("PTPRG", "PD153035", dec)).',
  'evidence(e_ksea("FYN", "PD153035", dec)).',
  'evidence(e_ksea("PTK6", "PD153035", dec)).'],
 ['evidence(e_ksea("SRC", "PD153035", inc)).',
  'evidence(e_ksea("PTPRR", "PD153035", inc)).'],
 ['evidence(e_ksea("ABL1", "PD153035", inc)).'],
 ['evidence(e_ksea("ABL1", "Tofacitinib", inc)).']]

In [17]:
inference_output = 'models/ebdt_data/sub_network/p_model/inference_log_10.txt'
inf_data_dict = inf.infer_marginf(model_inference, queries, inference_output, evidence=evidence_list)
pickle.dump(inf_data_dict, open('models/ebdt_data/sub_network/p_model/testing_releev_exp10_200i.pkl', 'wb'))

['query(p_occupancy("ABI1(Y213)", "PD153035", _)).']
1/117
Inferring data (['query(p_occupancy("ABI1(Y213)", "PD153035", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp10_testing_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_10.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0000s
[INFO] Grounding: 0.0226s
[INFO] Cycle breaking: 0.0018s
[INFO] Clark's completion: 0.0003s
[INFO] DSharp compilation: 0.0212s
[INFO] Total time: 2.6204s
Finished inference (['query(p_occupancy("ABI1(Y213)", "PD153035", _)).'])...
['query(p_occupancy("ANXA2P2(Y24)", "PD153035", _)).']
2/117
Inferring data (['query(p_occupancy("ANXA2P2(Y24)", "PD153035", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp10_testing_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_10.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0000s
[INFO] Grounding: 0.0183s
[INFO] Cycle breaking: 0.0023s
[INFO] Clark's complet